![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2Farchitectures%2Ftracking%2Fsetup%2Fgithub&dt=GitHub+Metrics+-+2+-+Traffic+-+Reporting+Scheduled+Query.ipynb)

# GitHub Metrics: Traffic History Reporting

This notebook shows the processing of raw data in the dataset `github_metrics` into a format ready for reporting stored in the `reporting` folder.  The queries develped here are scheduled in the Cloud Function create by the step 3 notebook for Commits.

**Source Dataset** 
- `vertex-ai-mlops-369716.github_metrics`
- **Source Tables**
    - `traffic_clones`
    - `traffic_popular_paths`
    - `traffic_popular_referrers`
    - `traffic_views`
    - `stargazers`
    - `forks`
    - `subscribers`

**Destination Dataset** 
- `vertex-ai-mlops-369716.reporting`
- **Destination Tables/Views**
    - `traffic_clones`
    - `traffic_popular_paths`
    - `traffic_popular_referrers`
    - `traffic_views`
    - `stargazers`
    - `forks`
    - `subscribers`

Alternative Way To Schedule is [BigQuery Scheduled Queries](https://cloud.google.com/bigquery/docs/scheduling-queries#set_up_scheduled_queries)


---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/github/GitHub%20Metrics%20-%202%20-%20Traffic%20-%20Reporting%20Scheduled%20Query.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [ ]:
PROJECT_ID = 'vertex-ai-mlops-369716' # replace with project ID

In [ ]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

Updated property [core/project].


---
## Setup

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'vertex-ai-mlops-369716'

In [ ]:
BQ_PROJECT = PROJECT_ID

In [ ]:
from datetime import datetime, timedelta
from google.cloud import bigquery

In [ ]:
bq = bigquery.Client(project = PROJECT_ID)

---
## Inital Reporting Tables

For these tables it is mostly a conversion of data types for time columns and copy over to reporting.

### traffic_clones


In [ ]:
BQ_TABLE = 'traffic_clones'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
ORDER BY timestamp
"""
job = bq.query(query = query)
job.result()

### traffic_popular_paths

In [ ]:
BQ_TABLE = 'traffic_popular_paths'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
ORDER BY timestamp, count DESC
"""
job = bq.query(query = query)
job.result()

### traffic_popular_referrers

In [ ]:
BQ_TABLE = 'traffic_popular_referrers'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
ORDER BY timestamp, count DESC
"""
job = bq.query(query = query)
job.result()

### traffic_views

In [ ]:
BQ_TABLE = 'traffic_views'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
ORDER BY timestamp
"""
job = bq.query(query = query)
job.result()

### stargazers

In [ ]:
BQ_TABLE = 'stargazers'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE added >= dropped
ORDER BY login
"""
job = bq.query(query = query)
job.result()

### forks

In [ ]:
BQ_TABLE = 'forks'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE added >= dropped
ORDER BY full_name
"""
job = bq.query(query = query)
job.result()

### subscribers

In [ ]:
BQ_TABLE = 'subscribers'
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE added >= dropped
ORDER BY login
"""
job = bq.query(query = query)
job.result()

---
## Incremental Updates


In [ ]:
yesterday = (datetime.now() - timedelta(days = 1)).strftime("%Y-%m-%dT00:00:00Z")
yesterday

'2023-02-27T00:00:00Z'

### traffic_clones
Check for updates to most recent records and any newer records based on `timestamp`

In [ ]:
BQ_TABLE = 'traffic_clones'
query_1 = f"""
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE timestamp >= '{yesterday}'
ORDER BY timestamp;
"""
bq.query(query = query_1).to_dataframe()

count  uniques  uniques_last14days                      repo  timestamp
0     13        9                66.0  statmike/vertex-ai-mlops 2023-02-27
1      0        0                67.0  statmike/vertex-ai-mlops 2023-02-28

In [ ]:
query_1 = f"""
DELETE FROM `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` WHERE timestamp >= DATETIME(TIMESTAMP('{yesterday}'));
INSERT INTO `{BQ_PROJECT}.reporting.github_{BQ_TABLE}`{query_1}
"""
print(query_1)


DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_clones` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `vertex-ai-mlops-369716.reporting.github_traffic_clones`
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `vertex-ai-mlops-369716.github_metrics.traffic_clones`
WHERE timestamp >= '2023-02-27T00:00:00Z'
ORDER BY timestamp;




In [ ]:
job = bq.query(query = query_1)
job.result()
job.state

'DONE'

### traffic_popular_paths
Check for any newer records based on `timestamp`

In [ ]:
BQ_TABLE = 'traffic_popular_paths'
query_2 = f"""
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE timestamp >= '{yesterday}'
ORDER BY timestamp, count DESC;
"""
bq.query(query = query_2).to_dataframe()

count  uniques                                               file  \
0     609      221                /statmike/vertex-ai-mlops/readme.md   
1      68       36  /statmike/vertex-ai-mlops/04 - scikit-learn/re...   
2      60       38  /statmike/vertex-ai-mlops/00 - Setup/00 - Envi...   
3      59       37     /statmike/vertex-ai-mlops/00 - Setup/readme.md   
4      58       34  /statmike/vertex-ai-mlops/02 - Vertex AI AutoM...   
5      58       26  /statmike/vertex-ai-mlops/01 - Data Sources/01...   
6      50       32  /statmike/vertex-ai-mlops/05 - TensorFlow/read...   
7      39       25  /statmike/vertex-ai-mlops/01 - Data Sources/re...   
8      37       22  /statmike/vertex-ai-mlops/03 - BigQuery ML (BQ...   
9      32       16  /statmike/vertex-ai-mlops/architectures/overvi...   
10    600      222                /statmike/vertex-ai-mlops/readme.md   
11     60       35  /statmike/vertex-ai-mlops/04 - scikit-learn/re...   
12     60       36  /statmike/vertex-ai-mlops/00 - Setup/00 - Envi...   
13     59       23  /statmike/vertex-ai-mlops/01 - Data Sources/01...   
14     56       33     /statmike/vertex-ai-mlops/00 - Setup/readme.md   
15     55       33  /statmike/vertex-ai-mlops/02 - Vertex AI AutoM...   
16     49       31  /statmike/vertex-ai-mlops/05 - TensorFlow/read...   
17     37       23  /statmike/vertex-ai-mlops/01 - Data Sources/re...   
18     32       16  /statmike/vertex-ai-mlops/architectures/overvi...   
19     32       19  /statmike/vertex-ai-mlops/03 - BigQuery ML (BQ...   

                        repo  timestamp  
0   statmike/vertex-ai-mlops 2023-02-27  
1   statmike/vertex-ai-mlops 2023-02-27  
2   statmike/vertex-ai-mlops 2023-02-27  
3   statmike/vertex-ai-mlops 2023-02-27  
4   statmike/vertex-ai-mlops 2023-02-27  
5   statmike/vertex-ai-mlops 2023-02-27  
6   statmike/vertex-ai-mlops 2023-02-27  
7   statmike/vertex-ai-mlops 2023-02-27  
8   statmike/vertex-ai-mlops 2023-02-27  
9   statmike/vertex-ai-mlops 2023-02-27  
10  statmike/vertex-ai-mlops 2023-02-28  
11  statmike/vertex-ai-mlops 2023-02-28  
12  statmike/vertex-ai-mlops 2023-02-28  
13  statmike/vertex-ai-mlops 2023-02-28  
14  statmike/vertex-ai-mlops 2023-02-28  
15  statmike/vertex-ai-mlops 2023-02-28  
16  statmike/vertex-ai-mlops 2023-02-28  
17  statmike/vertex-ai-mlops 2023-02-28  
18  statmike/vertex-ai-mlops 2023-02-28  
19  statmike/vertex-ai-mlops 2023-02-28

In [ ]:
query_2 = f"""
DELETE FROM `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` WHERE timestamp >= DATETIME(TIMESTAMP('{yesterday}'));
INSERT INTO `{BQ_PROJECT}.reporting.github_{BQ_TABLE}`{query_2}
"""
print(query_2)


DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_popular_paths` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `vertex-ai-mlops-369716.reporting.github_traffic_popular_paths`
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `vertex-ai-mlops-369716.github_metrics.traffic_popular_paths`
WHERE timestamp >= '2023-02-27T00:00:00Z'
ORDER BY timestamp, count DESC;




In [ ]:
job = bq.query(query = query_2)
job.result()
job.state

'DONE'

### traffic_popular_referrers
Check for any newer records based on `timestamp`

In [ ]:
BQ_TABLE = 'traffic_popular_referrers'
query_3 = f"""
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE timestamp >= '{yesterday}'
ORDER BY timestamp, count DESC;
"""
bq.query(query = query_3).to_dataframe()

referrer  count  uniques                      repo  \
0                       youtube.com    431      110  statmike/vertex-ai-mlops   
1                        github.com    238       31  statmike/vertex-ai-mlops   
2                            Google    214       58  statmike/vertex-ai-mlops   
3      statics.teams.cdn.office.net     12        3  statmike/vertex-ai-mlops   
4   notebooks.githubusercontent.com     11        6  statmike/vertex-ai-mlops   
5                     m.youtube.com      3        2  statmike/vertex-ai-mlops   
6                   mail.google.com      2        2  statmike/vertex-ai-mlops   
7         colab.research.google.com      1        1  statmike/vertex-ai-mlops   
8                       youtube.com    408      106  statmike/vertex-ai-mlops   
9                        github.com    255       33  statmike/vertex-ai-mlops   
10                           Google    204       55  statmike/vertex-ai-mlops   
11     statics.teams.cdn.office.net     12        3  statmike/vertex-ai-mlops   
12  notebooks.githubusercontent.com     11        6  statmike/vertex-ai-mlops   
13                    m.youtube.com      4        2  statmike/vertex-ai-mlops   
14                  mail.google.com      2        2  statmike/vertex-ai-mlops   
15        colab.research.google.com      1        1  statmike/vertex-ai-mlops   

    timestamp  
0  2023-02-27  
1  2023-02-27  
2  2023-02-27  
3  2023-02-27  
4  2023-02-27  
5  2023-02-27  
6  2023-02-27  
7  2023-02-27  
8  2023-02-28  
9  2023-02-28  
10 2023-02-28  
11 2023-02-28  
12 2023-02-28  
13 2023-02-28  
14 2023-02-28  
15 2023-02-28

In [ ]:
query_3 = f"""
DELETE FROM `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` WHERE timestamp >= DATETIME(TIMESTAMP('{yesterday}'));
INSERT INTO `{BQ_PROJECT}.reporting.github_{BQ_TABLE}`{query_3}
"""
print(query_3)


DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_popular_referrers` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `vertex-ai-mlops-369716.reporting.github_traffic_popular_referrers`
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `vertex-ai-mlops-369716.github_metrics.traffic_popular_referrers`
WHERE timestamp >= '2023-02-27T00:00:00Z'
ORDER BY timestamp, count DESC;




In [ ]:
job = bq.query(query = query_3)
job.result()
job.state

'DONE'

### traffic_views
Check for updates to most recent records and any newer records based on `timestamp`

In [ ]:
BQ_TABLE = 'traffic_views'
query_4 = f"""
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE timestamp >= '{yesterday}'
ORDER BY timestamp;
"""
bq.query(query = query_4).to_dataframe()

count  uniques  uniques_last14days                      repo  timestamp
0    132       44               297.0  statmike/vertex-ai-mlops 2023-02-27
1     79       20               292.0  statmike/vertex-ai-mlops 2023-02-28

In [ ]:
query_4 = f"""
DELETE FROM `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` WHERE timestamp >= DATETIME(TIMESTAMP('{yesterday}'));
INSERT INTO `{BQ_PROJECT}.reporting.github_{BQ_TABLE}`{query_4}
"""
print(query_4)


DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_views` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `vertex-ai-mlops-369716.reporting.github_traffic_views`
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `vertex-ai-mlops-369716.github_metrics.traffic_views`
WHERE timestamp >= '2023-02-27T00:00:00Z'
ORDER BY timestamp;




In [ ]:
job = bq.query(query = query_4)
job.result()
job.state

'DONE'

### stargazers
Replace - but only if a change occured

In [ ]:
BQ_TABLE = 'stargazers'
query_5 = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE added >= dropped
ORDER BY login;
"""

In [ ]:
print(query_5)


CREATE OR REPLACE TABLE `vertex-ai-mlops-369716.reporting.github_stargazers` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `vertex-ai-mlops-369716.github_metrics.stargazers`
WHERE added >= dropped
ORDER BY login;



In [ ]:
job = bq.query(query = query_5)
job.result()
job.state

'DONE'

### forks
Replace - but only if a change occured

In [ ]:
BQ_TABLE = 'forks'
query_6 = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE added >= dropped
ORDER BY full_name;
"""

In [ ]:
print(query_6)


CREATE OR REPLACE TABLE `vertex-ai-mlops-369716.reporting.github_forks` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `vertex-ai-mlops-369716.github_metrics.forks`
WHERE added >= dropped
ORDER BY full_name;



In [ ]:
job = bq.query(query = query_6)
job.result()
job.state

'DONE'

### subscribers
Replace - but only if a change occured

In [ ]:
BQ_TABLE = 'subscribers'
query_7 = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.reporting.github_{BQ_TABLE}` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `{BQ_PROJECT}.github_metrics.{BQ_TABLE}`
WHERE added >= dropped
ORDER BY login;
"""

In [ ]:
print(query_7)


CREATE OR REPLACE TABLE `vertex-ai-mlops-369716.reporting.github_subscribers` AS
SELECT * EXCEPT(added, dropped),
    CASE WHEN added = '' THEN NULL ELSE DATETIME(TIMESTAMP(added)) END AS added,
    CASE WHEN dropped = '' THEN NULL ELSE DATETIME(TIMESTAMP(dropped)) END AS dropped
FROM `vertex-ai-mlops-369716.github_metrics.subscribers`
WHERE added >= dropped
ORDER BY login;



In [ ]:
job = bq.query(query = query_7)
job.result()
job.state

'DONE'

## Query To Schedule

In the notebook 'GitHub Metrics - 3 - Traffic - Incremental Update Cloud Function.ipynb' the cloud function that updates the raw data in the dataset `github_metrics` is setup.  Since updating the reporting should come right after the raw data update it makes sense to add the updating queries to that Cloud Function.  The query is constructed by the print statement below and then copy/pasted to the the cloud function for daily execution.

>An alternative way to schedule a query is using [BigQuery's Scheduled Queries](https://cloud.google.com/bigquery/docs/scheduling-queries) capability.  This has two ways to get started within the console:
>- From the query editor there is a Schedule option in the Query Editor Tool Bar
>- From the BigQuery > Scheduled Queries > + Create Scheduled Query In Editor


In [ ]:
query = query_1 + ';\n' + query_2 + ';\n' + query_3 + ';\n' + query_4 + ';\n' + query_5 + ';\n' + query_6 + ';\n' + query_7 + ';'
print(query)


DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_clones` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `vertex-ai-mlops-369716.reporting.github_traffic_clones`
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `vertex-ai-mlops-369716.github_metrics.traffic_clones`
WHERE timestamp >= '2023-02-27T00:00:00Z'
ORDER BY timestamp;

;

DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_popular_paths` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `vertex-ai-mlops-369716.reporting.github_traffic_popular_paths`
SELECT * EXCEPT(timestamp),
    DATETIME(TIMESTAMP(timestamp)) AS timestamp
FROM `vertex-ai-mlops-369716.github_metrics.traffic_popular_paths`
WHERE timestamp >= '2023-02-27T00:00:00Z'
ORDER BY timestamp, count DESC;

;

DELETE FROM `vertex-ai-mlops-369716.reporting.github_traffic_popular_referrers` WHERE timestamp >= DATETIME(TIMESTAMP('2023-02-27T00:00:00Z'));
INSERT INTO `v